# Обучение модели

## Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf

from IPython.display import display
from tensorflow import keras
from tensorflow.keras.layers import Dense

import sys
sys.path.append('custom_modules')

import Data_processing as dp
import Data_visualization as dv

## Константы 

In [2]:
path_to_data = "Data\Обработанные данные"

In [3]:
# детерминация случайных величин, отвечающих за выбор первоначальных весов и биасов
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)

In [4]:
# всякие константы для последующей работы

#///////////////////////////////// для взятия данных из файлов

path_to_data = 'ngp.csv' # путь к файлу, из которого берутся данные для обучения
target = 'price' # название взятой величины из файла

#///////////////////////////////// для создания слоев

CRT_hidden_layer_act_fun = 'relu'   # функция активация скрытых слоев и входного
CRT_output_layer_act_fun = 'sigmoid' # функция активация выходного слоя

CRT_dict = {4: CRT_hidden_layer_act_fun, 
            400: CRT_hidden_layer_act_fun,
            20: CRT_hidden_layer_act_fun,
            1: CRT_output_layer_act_fun} # size(layer) plus activation func

#///////////////////////////////// для компиляции 

CMP_learning_rate = 0.00005 # шаг сходимости back propogation
CMP_solver = keras.optimizers.Adam(CMP_learning_rate) # оптимизатор
CMP_loss_func = 'mean_squared_error'# функция потерь

#///////////////////////////////// для колбэков

    # для Early_stopping
ES_patience = 15 # кол-во эпох без улучшений
ES_min_delta = 0.0001 # минимальное улучшение параметра за cur_patience
ES_monitor_parametr =  'loss' # отслеживаемый параметр 
ES_save_best_weights = True # сохранять ли веса нейронки с лучшими результатами
    
    # для ReduceLROnPlateau
RLPOP_monitor_parametr = 'val_loss'  # отслеживаемый параметр 
RLPOP_factor = 0.1 # множитель для расчета нового шага сходимости (new_learning_rate = old_learning_rate*RLPOP_factor)
RLPOP_patience = 10 # кол-во эпох без улучшений
RLPOP_verbose = 1 # выводить ли прогресс изменения шага сходимости в его процессее
RLPOP_mode = 'auto' # выбирает, уменьшать шаг сходимости при росте величины или при её уменьшении
RLPOP_min_delta = 0.0001 # порог изменения отслеживаемого значения
RLPOP_cooldown = 0 # количество эпох до возобновления работы после изменения шага сходимости
RLPOP_min_lr = 0 # минимальное значение шага сходимости

    # для CallbackList
CBL_add_history = True # вызывать ли колбэк History (если он не был довавлен вручную)
CBL_add_progbar = True # вызывать ли колбэк ProgbarLogger (если он не был довавлен вручную)
    
#///////////////////////////////// для тренировки

FIT_batch_size = 4 #13, 4 # размер bach при обучении/тестировании
FIT_shuffle = True # перемешивать ли данные
FIT_verbose = True # выводить ли прогресс обучения в его процессее
FIT_epochs = 50 # количество эпох обучения
FIT_validation_split = 0.2 # процент валидационных данных, отсекаемых из тестовой выборки

In [5]:
# создание архитектуры нейронки
model = keras.Sequential()

for x in CRT_dict.items():
    model.add(Dense(x[0], activation = x[1]))
    
model.compile(loss = CMP_loss_func, optimizer = CMP_solver)

In [6]:
# Создание и настройка колбэков
callback_list = [] # массив колбэков до подачи в колбек "callbacklist"

temp = keras.callbacks.EarlyStopping(
            monitor = ES_monitor_parametr, 
            min_delta = ES_min_delta, 
            patience = ES_patience,
            restore_best_weights = ES_save_best_weights
            )
callback_list.append(temp)

temp = keras.callbacks.ReduceLROnPlateau(
            monitor = RLPOP_monitor_parametr, 
            factor = RLPOP_factor, 
            patience = RLPOP_patience, 
            verbose = RLPOP_verbose,
            mode = RLPOP_mode, 
            min_delta = RLPOP_min_delta, 
            cooldown = RLPOP_cooldown, 
            min_lr = RLPOP_min_lr
            )
callback_list.append(temp)

FIT_callback_list = keras.callbacks.CallbackList(
            callbacks = callback_list, 
            add_history = CBL_add_history, 
            add_progbar = CBL_add_progbar, 
            model = model
            )

In [7]:
def prepare_df(df: pd.DataFrame):
    X = pd.DataFrame({}, columns=df.columns)
    Y = pd.DataFrame({}, columns=df.columns)

    for row_index,row in df.iterrows():
        if (row_index % 5 == 0):
            Y = pd.concat([Y, pd.DataFrame(row).T], axis=0)
        else:
            X = pd.concat([X, pd.DataFrame(row).T], axis=0)

In [21]:
d = {'col1': range(10),
     'col2': range(10,20),
     'col3': range(20,30),
     'col4': range(20,30)}

df = pd.DataFrame(data = d)

#display(df)

X = pd.DataFrame({}, columns=df.columns)
Y = pd.DataFrame({}, columns=df.columns)
temp = pd.DataFrame({}, columns=df.columns)

for row_index,row in df.iterrows():
    if (row_index % 5 == 0):
        
        new_row = list(zip(df.columns, df.iloc[row_index+1:row_index+5])) #{col: cell for col in df.columns for cell in [1,2,3,4]}
    
        new_row = {col: cell for col, cell in new_row}
        
        print(new_row)
        
        #row_index+=5
        #X = X.append(new_row, ignore_index=True)
        Y = pd.concat([Y, pd.DataFrame(row).T], axis=0)
    else:
        temp = pd.concat([temp, pd.DataFrame(row).T], axis=0)

{'col1': 1, 'col2': 11, 'col3': 21, 'col4': 21}
{'col1': 1, 'col2': 11, 'col3': 21, 'col4': 21}


In [9]:
#print("X", type(X), X.shape)
print("temp", type(temp), temp.shape)
print("Y", type(Y), Y.shape)
#display(X)
display(temp)
display(Y)

temp <class 'pandas.core.frame.DataFrame'> (8, 4)
Y <class 'pandas.core.frame.DataFrame'> (2, 4)


,col1,col2,col3,col4
1,1,11,21,21
2,2,12,22,22
3,3,13,23,23
4,4,14,24,24
6,6,16,26,26
7,7,17,27,27
8,8,18,28,28
9,9,19,29,29


,col1,col2,col3,col4
0,0,10,20,20
5,5,15,25,25


In [10]:
# подготовка данных

# извлечь датасет из файла
#first_df = dp.get_df(path_to_data)

# создать тестовый/тренировочный датасет
'''prepared_df = prepare_df(cols,names)'''

# создать тестовые/тренировочные данные из тестового датасета
#prepared_data = prep_data_4_1_time(cols, names, prepared_df)

#X_train,Y_train,X_test,Y_test = prepared_data

'prepared_df = prepare_df(cols,names)'

In [11]:
# тренировка модели
'''history = model.fit(X_train, 
                    Y_train, 
                    batch_size = FIT_batch_size, 
                    epochs = FIT_epochs, 
                    verbose = FIT_verbose, 
                    validation_split = FIT_validation_split, 
                    shuffle = FIT_shuffle, 
                    callbacks = FIT_callback_list)'''

'history = model.fit(X_train, \n                    Y_train, \n                    batch_size = FIT_batch_size, \n                    epochs = FIT_epochs, \n                    verbose = FIT_verbose, \n                    validation_split = FIT_validation_split, \n                    shuffle = FIT_shuffle, \n                    callbacks = FIT_callback_list)'

In [12]:
# Вывод графика изменения ошибки
'''plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'], linestyle = '--')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.show()'''

"plt.plot(history.history['loss'])\nplt.plot(history.history['val_loss'], linestyle = '--')\nplt.title('model loss')\nplt.ylabel('loss')\nplt.xlabel('epoch')\nplt.legend(['train', 'val'], loc = 'upper left')\nplt.show()"

In [13]:
# тест модели

'''model.evaluate(X_test, Y_test, batch_size = FIT_batch_size)'''

'model.evaluate(X_test, Y_test, batch_size = FIT_batch_size)'

In [14]:
'''model.save("NetWork_4_to_4")'''

'model.save("NetWork_4_to_4")'

In [15]:
#посмотреть сравнение оригинальных и обработанных данных

'''print("/////////////////////")
print("Difference between prepared data and original ones\n\n")
for address, dirs, files in os.walk(new_path_to_data):
    if(dirs == []):
        temp_old_path = "\\".join([old_path_to_data, address.split(sep="\\")[-1]])
        
        print("Current dirs:")
        print("Old_Addres: ", temp_old_path)
        print("New_Addres: ", address)
        #print("Dirs: ", dirs)
        #print("Files: ", files)
        
        for file in files:
            print("\n\t\tDifference between:")
            print("\t","\\".join([temp_old_path, file]))
            print("\t","\\".join([address, file]))
            #orig_df = dp.get_df("\\".join([temp_old_path, file]))
            #prepared_df = dp.get_df("\\".join([address, file]))
            #dv.draw_4_graphs_with_overlay_for_2_df(orig_df, prepared_df)
            
        print("/////////////////////")'''

'print("/////////////////////")\nprint("Difference between prepared data and original ones\n\n")\nfor address, dirs, files in os.walk(new_path_to_data):\n    if(dirs == []):\n        temp_old_path = "\\".join([old_path_to_data, address.split(sep="\\")[-1]])\n        \n        print("Current dirs:")\n        print("Old_Addres: ", temp_old_path)\n        print("New_Addres: ", address)\n        #print("Dirs: ", dirs)\n        #print("Files: ", files)\n        \n        for file in files:\n            print("\n\t\tDifference between:")\n            print("\t","\\".join([temp_old_path, file]))\n            print("\t","\\".join([address, file]))\n            #orig_df = dp.get_df("\\".join([temp_old_path, file]))\n            #prepared_df = dp.get_df("\\".join([address, file]))\n            #dv.draw_4_graphs_with_overlay_for_2_df(orig_df, prepared_df)\n            \n        print("/////////////////////")'